In [4]:
from __future__ import print_function
import math
from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import metrics
from tensorflow.python.data import Dataset
tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

In [5]:
df = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=',')

We will randomize the data for the sake of SGD.<br/>
Additionally we will scale 'median_house_value' to be in units of thousands.

In [6]:
df = df.reindex(np.random.permutation(df.index))
df['median_house_value'] /= 1000

In [8]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
2564,-117.7,34.1,40.0,1609.0,258.0,624.0,242.0,5.5,158.2
2032,-117.3,34.1,52.0,802.0,160.0,564.0,131.0,2.2,63.5
4929,-118.1,33.8,36.0,1946.0,346.0,871.0,336.0,5.2,254.8
7325,-118.3,33.9,36.0,1342.0,259.0,706.0,261.0,4.2,236.6
11090,-121.0,37.7,46.0,1750.0,347.0,754.0,356.0,2.9,106.0


In [9]:
df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0
mean,-119.6,35.6,28.6,2643.7,539.4,1429.6,501.2,3.9,207.3
std,2.0,2.1,12.6,2179.9,421.5,1147.9,384.5,1.9,116.0
min,-124.3,32.5,1.0,2.0,1.0,3.0,1.0,0.5,15.0
25%,-121.8,33.9,18.0,1462.0,297.0,790.0,282.0,2.6,119.4
50%,-118.5,34.2,29.0,2127.0,434.0,1167.0,409.0,3.5,180.4
75%,-118.0,37.7,37.0,3151.2,648.2,1721.0,605.2,4.8,265.0
max,-114.3,42.0,52.0,37937.0,6445.0,35682.0,6082.0,15.0,500.0


## Step 1: Define Features and Configure Feature Columns

In [11]:
# define the input feature
my_feature = df[['total_rooms']]
# configure as numerical feature
feature_columns = [tf.feature_column.numeric_column('total_rooms')]

## Step 2: Define the targets

In [12]:
targets = df['median_house_value']

##  Step 3: Configure the LinearRegressor

In [13]:
# use Gradient Descent as the optomizer to train the model
my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0000001)
my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer,5.0)

In [14]:
# configure the linear regression model with our feature columns and optimizer
linear_regressor = tf.estimator.LinearRegressor(
        feature_columns=feature_columns,
        optimizer=my_optimizer
)

## Step 4: Define the input function 